# Playing around with this notebook https://github.com/rowandl/portfolio/blob/master/Webscraping%20Indeed.com/Webscraping%20Indeed.ipynb

# Web Scraping for Indeed.com & Predicting Salaries

In this project, I will practice two major skills: collecting data by scraping a website and then building a binary classifier.

I am going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job I will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information, being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

I will convert this problem into classification and use a random forest classifier, as well as another classifier. 

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

I will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [20]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib
from urllib import urlopen
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [3]:
## YOUR CODE HERE
html = urllib.urlopen(URL).read()
soup = BeautifulSoup(html, "html.parser")
len(soup)

4

Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Functions to extract each item: location, company, job, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```


In [4]:
dflocation = pd.DataFrame(columns=["location"])
dfcompany = pd.DataFrame(columns=["company"])
dfjob_title = pd.DataFrame(columns=["job_title"])
dfsalary = pd.DataFrame(columns=["salary"])

#result.find(‘a’,attrs={‘data-tn-element’:‘jobTitle’}).text
def extract_location(result):
    for b in result.find_all('span', {'class': 'location'}):
        location = b.text
        dflocation.loc[len(dflocation)] = [location]    
        
def extract_company(result):        
    for i in result.find_all('span', {'class':'company'}):
        company = i.text
        dfcompany.loc[len(dfcompany)] = [company]   

def extract_job_title(result):
    for a in result.find_all('a', {'data-tn-element':'jobTitle'}):
        job_title = a.text
        dfjob_title.loc[len(dfjob_title)] = [job_title]

def extract_salary(result):
    for entry in result.find_all('td', {'class' : 'snip'}):
        try:
            salary = entry.find('nobr').text
            dfsalary.loc[len(dfsalary)] = [salary]  
        except:
            salary = 'NA'
            dfsalary.loc[len(dfsalary)] = [salary]    


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.
- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the l=New+York and the start=10. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).


In [50]:
#making a list of cities to run through
# 'New+York%2C+NY'
cities = ['California', 'Texas', 'New+York+State']

In [225]:
#making a dataframe which collects all the information from my webscraping
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
query = "data+scientist"
url_template= "https://www.indeed.com/jobs?q={}&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value to generate more results. 

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary'])
def validator(elem):
    if elem:
        return elem.text
    else:
        return 'NA'

for city in cities:
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(query,city,start)
        html = urllib.urlopen(url).read()
        soups = BeautifulSoup(html, "html.parser")
#         with open('somefile.html', 'a') as the_file:
#             the_file.write(str(soups))
        rows = soups.find_all('div', attrs = {'class':'row'})
        for b in rows:
#             print(b)
#             location = validator(b.find('div', attrs = {'class': 'location'}))
            location = city
            job_title = validator(b.find('a', attrs = {'data-tn-element':'jobTitle'}))
            url = b.find('a', attrs = {'data-tn-element':'jobTitle'}).href
            company = validator(b.find('span', attrs = {'class':'company'}))
            salary = validator(b.find(name="span", attrs={"class":"no-wrap"}))
            summaries=""
            spans = b.findAll('span', attrs={'class': 'summary'})
            for span in spans:
                summaries += span.text.strip()
            df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary, "summary": summaries, "url": url}, ignore_index=True) 
   

In [ ]:
df.drop_duplicates(inplace=True) #dropping duplicates
df.company.replace(regex=True,inplace=True,to_replace=["\n", "\r"],value="") #getting rid of /n in company
df.salary.replace(regex=True, inplace=True, to_replace=["\n", "\r", "\$"], value="") #getting rid of $ in salary
df.summary.replace(regex=True, inplace=True, to_replace=['\.\.\.'], value="") #getting rid of elipses in summary
df.summary[0]

In [182]:
#sending it to csvs to save the data
df.to_csv("~/Desktop/machineLearning.csv" , sep=',', encoding='utf-8')
df.shape

(369, 5)

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [183]:
#reading in the data
df1 = pd.read_csv('~/Desktop/machineLearning.csv', index_col=0)
# df2 = pd.read_csv('~/Desktop/GA/April11.csv')
# df3 = pd.read_csv('~/Desktop/GA/April12.csv')
# df4 = pd.read_csv('~/Desktop/GA/April1_11.csv')
# df5 = pd.read_csv('~/Desktop/GA/April2_11.csv')
# df6 = pd.read_csv('~/Desktop/GA/April_10.csv')
# df7 = pd.read_csv('~/Desktop/GA/April_11.csv')

In [184]:
data = pd.concat([df1]) #making into one df

In [185]:
def eda(dataframe): #code chunk to check quality of data
    print "missing values \n", dataframe.isnull().sum() #shows total amount of null values for each column
    print "dataframe types \n", dataframe.dtypes
    print "dataframe shape \n", dataframe.shape     
    print "dataframe describe \n", dataframe.describe()
    print "dataframe length =", len(dataframe) #length of the dataframe
    print "duplicates", dataframe.duplicated().sum() # added this to duplicates in the data
    for item in dataframe:
        print item
        print dataframe[item].nunique()

eda(data)

missing values 
location       0
company        0
job_title      0
salary       326
summary        0
dtype: int64
dataframe types 
location     object
company      object
job_title    object
salary       object
summary      object
dtype: object
dataframe shape 
(369, 5)
dataframe describe 
          location         company       job_title  \
count          369             369             369   
unique           3             303             215   
top     California          Google  Data Scientist   
freq           141               8             123   

                                          salary  \
count                                         43   
unique                                        38   
top                     110,000 - 160,000 a year   
freq                                           2   

                                                  summary  
count                                                 369  
unique                                                345

Lastly, I need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

I didn't think it was safe to multiply data for hour/week/month so I only wanted yearly data.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly, remove duplicate entries

In [186]:
salaries = data[data.salary.notnull()]
salaries = salaries[salaries.salary.str.contains('year')] #only getting yearly salaries
#http://stackoverflow.com/questions/15325182/how-to-filter-rows-in-pandas-by-regex
salaries.describe()
data.dropna(axis="index",subset=['salary'])
# data[data.salary.notnull()].groupby(['salary'])['location']

,location,company,job_title,salary,summary
5,California,Highlands Point Apartments,Data Scientist,"110,000 - 140,000 a year","If you are a Data Scientist with experience, p..."
17,California,"Enrich Consulting, Inc.",Quantitative Modeler / Coder - Analyst,"70,000 a year","As an Analyst, you will be customizing impleme..."
24,California,Negocios IT,Data scientist,"130,000 - 150,000 a year",*Required skills: 3-5 years’ of experience. ...
32,California,NESS USA INC,Data Scientist,"90,000 - 120,000 a year",Background in applied statistical modeling on ...
34,California,Protein Metrics Inc.,Customer Success Scientist,"120,000 - 140,000 a year",Our software allows scientists to use data the...
46,California,Stria,Data Scientist,"100,000 - 120,000 a year",We are looking for an accomplished data scient...
48,California,Intro Staffing,Software Engineer - Autonomous Driving / Self ...,"120,000 - 180,000 a year",Software Engineer - Autonomous Driving / Machi...
51,California,Lore IO,Python Developer,"130,000 - 170,000 a year","We are experts in search, analytics, machine l..."
66,California,FlowCommand Inc.,Data Scientist,"80,000 - 140,000 a year",Our sensors send ultrasonic data directly to o...
93,California,Blue Owl,Actuarial Pricing Analyst,"120,000 - 150,000 a year","We are a well funded team of elite developers,..."


In [187]:
#splitting my salaries to find an average
salaries.salary.replace(regex=True, inplace=True, to_replace="a year", value="")
salaries.salary.replace(regex=True, inplace=True, to_replace=",", value="")
salaries.salary.replace(regex=True, inplace=True, to_replace=" \(Indeed est.\)", value="")
salaries['salary_split'] = salaries['salary'].str.split('-')
salaries.head()
print(salaries.shape)

(40, 6)


In [188]:
#finding an average salary
def avg(salaries):
    salaries['lower'] = salaries['salary_split'].str[0].astype('float')
    salaries['upper'] = salaries['salary_split'].str[1].astype('float')
    salaries['avg'] = salaries[['lower','upper']].mean(axis=1)
avg(salaries)
salaries.head()

,location,company,job_title,salary,summary,salary_split,lower,upper,avg
5,California,Highlands Point Apartments,Data Scientist,110000 - 140000,"If you are a Data Scientist with experience, p...","[ 110000 , 140000 ]",110000.0,140000.0,125000.0
17,California,"Enrich Consulting, Inc.",Quantitative Modeler / Coder - Analyst,70000,"As an Analyst, you will be customizing impleme...",[ 70000 ],70000.0,NaN,70000.0
24,California,Negocios IT,Data scientist,130000 - 150000,*Required skills: 3-5 years’ of experience. ...,"[ 130000 , 150000 ]",130000.0,150000.0,140000.0
32,California,NESS USA INC,Data Scientist,90000 - 120000,Background in applied statistical modeling on ...,"[ 90000 , 120000 ]",90000.0,120000.0,105000.0
34,California,Protein Metrics Inc.,Customer Success Scientist,120000 - 140000,Our software allows scientists to use data the...,"[ 120000 , 140000 ]",120000.0,140000.0,130000.0


In [189]:
#dropping columns so I'm only left with average column
clean_sal = salaries.drop(['salary','salary_split', 'lower', 'upper'], axis=1)
clean_sal.head()
clean_sal.shape

(40, 5)

In [190]:
clean_sal['citystate'] = clean_sal['location'].str.split(',') #splitting the location to separate city and state
# clean_sal['city'] = clean_sal['citystate'].str[0] #getting cities
# clean_sal['state'] = clean_sal['citystate'].str[1] #getting states
clean_sal['city'] = clean_sal['location'] #getting cities
clean_sal['state'] = clean_sal['location'] #getting state
clean_sal['state'] = clean_sal['state'].str[0:3] #getting only 2 letter state codes
clean_sal.drop(['location','citystate'], axis=1, inplace=True) #dropping columns so I'm only left with cities/states
clean_sal.head()

,company,job_title,summary,avg,city,state
5,Highlands Point Apartments,Data Scientist,"If you are a Data Scientist with experience, p...",125000.0,California,Cal
17,"Enrich Consulting, Inc.",Quantitative Modeler / Coder - Analyst,"As an Analyst, you will be customizing impleme...",70000.0,California,Cal
24,Negocios IT,Data scientist,*Required skills: 3-5 years’ of experience. ...,140000.0,California,Cal
32,NESS USA INC,Data Scientist,Background in applied statistical modeling on ...,105000.0,California,Cal
34,Protein Metrics Inc.,Customer Success Scientist,Our software allows scientists to use data the...,130000.0,California,Cal


In [191]:
eda(clean_sal)

missing values 
company      0
job_title    0
summary      0
avg          0
city         0
state        0
dtype: int64
dataframe types 
company       object
job_title     object
summary       object
avg          float64
city          object
state         object
dtype: object
dataframe shape 
(40, 6)
dataframe describe 
                 avg
count      40.000000
mean   121123.000000
std     23055.263174
min     49920.000000
25%    109625.000000
50%    125000.000000
75%    135000.000000
max    164500.000000
dataframe length = 40
duplicates 0
company
37
job_title
25
summary
39
avg
26
city
3
state
3


### Save your results as a CSV

In [192]:
#saving my clean salary data to a csv
clean_sal.to_csv("~/Desktop/clean_salary.csv" , sep=',', encoding='utf-8')

## Predicting salaries using Random Forests + Another Classifier

In [193]:
clean_sal['avg'].describe()

count        40.000000
mean     121123.000000
std       23055.263174
min       49920.000000
25%      109625.000000
50%      125000.000000
75%      135000.000000
max      164500.000000
Name: avg, dtype: float64

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [194]:
#finding the median
import numpy as np
median = np.median(clean_sal.avg)
print(median)

print("Medians per city:")
print(clean_sal.groupby(['city'])['avg'].mean())

125000.0
Medians per city:
city
California        121666.666667
New+York+State    135423.076923
Texas             108294.666667
Name: avg, dtype: float64


In [195]:
c = pd.read_csv('~/Desktop/clean_salary.csv', index_col=0)

In [196]:
c['dumsal'] = (c["avg"] >= c["avg"].median()).astype(int)
#http://stackoverflow.com/questions/36637011/how-can-i-create-a-dummy-variable-in-python-with-a-condition-below-or-above-medi
c[c['state']=="New"]

,company,job_title,summary,avg,city,state,dumsal
317,Gust,Inference Data Scientist,Hands on experience in data science or analyti...,135000.0,New+York+State,New,1
338,kapil.s@progressivestaffing.net,Data Scientist,Technical proficiency with optimizing data col...,155500.0,New+York+State,New,1
345,Net-Consultants,Machine Learning Data Scientist,This is an exciting opportunity if you are int...,130000.0,New+York+State,New,1
357,Transfix.io,Data Scientist,"As a Data Scientist, you will implement machin...",147000.0,New+York+State,New,1
358,Tilting Point,Game Data Scientist,"Statistical, machine learning, and Bayesian kn...",122500.0,New+York+State,New,0
360,Simon Property Group,Venture Capital Analyst - Data Scientist,Simon Ventures specializes in investing in nex...,136000.0,New+York+State,New,1
361,JP Morgan Chase,Data Scientist - Global Research & Data Analyt...,The successful candidate will apply data analy...,164500.0,New+York+State,New,1
362,Outbrain Inc.,Data Scientist,Data Scientist (Sphere). Meaningful experience...,130000.0,New+York+State,New,1
365,Vettery,Data Scientist,Leveraging machine learning models that track ...,116500.0,New+York+State,New,0
366,NewYork-Presbyterian Hospital,Data Scientist,Our mission is to enhance healthcare delivery ...,133500.0,New+York+State,New,1


In [197]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [198]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
X = c.summary
y = c.dumsal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [199]:
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_train = pd.DataFrame(cvec.transform(X_train).todense(),
             columns=cvec.get_feature_names())

In [200]:
df_test = pd.DataFrame(cvec.transform(X_test).todense(),
                      columns=cvec.get_feature_names())
print df_train.shape
print y_train.shape
print df_test.shape
print y_test.shape

(26, 227)
(26,)
(14, 227)
(14,)


In [201]:
lr = LogisticRegression()
lr.fit(df_train, y_train)
lr.score(df_test, y_test)
#this is my title score

0.21428571428571427

In [205]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
cv = 7
# Perform 6-fold cross validation
scores = cross_val_score(lr, df_test, y_test, cv=cv)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(lr, df_test, y_test, cv=cv)
accuracy = metrics.accuracy_score(y_test, predictions)
print "Cross-Predicted Accuracy:", accuracy

Cross-validated scores: [ 1.   0.5  1.   0.   1.   0.   0.5]
Cross-Predicted Accuracy: 0.571428571429


In [206]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight='balanced')
RF = rf.fit(df_train,y_train)
s = cross_val_score(rf, df_train, y_train, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forrest", s.mean().round(3), s.std().round(3))

Random Forrest Score:	0.5 ± 0.045


In [207]:
rf_features = pd.DataFrame(RF.feature_importances_,
                                   index = df_train.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
rf_features.head(10)

,importance
scientist,0.061869
consumer,0.038589
exciting,0.038514
algorithms,0.037125
machine,0.034248
areas,0.029412
chosen,0.029344
millions,0.025356
plans,0.024385
business,0.023204


#### Create a few new variables in my dataframe to represent interesting features of a job title. Then I built a new Random Forest with these features. 

In [218]:
#manager, visualization, scientist, engineer, analyst, senior
vect_col = 'summary'
c['visualization'] = map(int, c[vect_col].str.lower().str.contains('vis'))
c['manager'] = map(int, c[vect_col].str.lower().str.contains('manager')|
                  c[vect_col].str.lower().str.contains('mngr')|
                  c[vect_col].str.lower().str.contains('mgr'))
c['scientist'] = map(int, c[vect_col].str.lower().str.contains('scien'))
c['associate'] = map(int, c[vect_col].str.lower().str.contains('asso'))
c['machine'] = map(int, c[vect_col].str.lower().str.contains('mach'))
c['learning'] = map(int, c[vect_col].str.lower().str.contains('lear'))
c['mining'] = map(int, c[vect_col].str.lower().str.contains('minin'))
c['engineer'] = map(int, c[vect_col].str.lower().str.contains('engi')|
                         c[vect_col].str.lower().str.contains('engr'))
c['senior'] = map(int, c[vect_col].str.lower().str.contains('sr')|
                  c[vect_col].str.lower().str.contains('snr')|
                  c[vect_col].str.lower().str.contains('senior'))
#making dummies for the words at the top of this cell
c.loc[:, c.columns != 'summary'].head()

,company,job_title,avg,city,state,dumsal,visualization,manager,scientist,associate,machine,learning,engineer,senior,mining
5,Highlands Point Apartments,Data Scientist,125000.0,California,Cal,1,0,0,1,0,0,0,0,0,0
17,"Enrich Consulting, Inc.",Quantitative Modeler / Coder - Analyst,70000.0,California,Cal,0,0,0,0,0,0,0,0,0,0
24,Negocios IT,Data scientist,140000.0,California,Cal,1,0,0,0,0,0,0,0,0,0
32,NESS USA INC,Data Scientist,105000.0,California,Cal,0,0,0,0,0,0,0,0,0,0
34,Protein Metrics Inc.,Customer Success Scientist,130000.0,California,Cal,1,0,0,1,0,0,0,0,0,0


In [220]:
#completing a logistic regression model
from sklearn.linear_model import LogisticRegression
X = c[['manager', 'visualization', 'scientist', 'engineer', 'senior', 'associate', 'mining']]
y = c.dumsal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)
#this is my concatonated score since it's worse I'm only going to use  

0.21428571428571427

In [221]:
#and a random forest classifier
rf = RandomForestClassifier(class_weight='balanced')
s = cross_val_score(rf, X, y, n_jobs=-1)
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forrest", s.mean().round(3), s.std().round(3))

Random Forrest Score:	0.575 ± 0.032


In [1]:
from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics
# Perform 6-fold cross validation
scores = cross_val_score(rf, X, y, cv=6)
print "Cross-validated scores:", scores
# Make cross validated predictions
predictions = cross_val_predict(rf, X, y, cv=6)
accuracy = metrics.accuracy_score(y, predictions)
print "Cross-Predicted Accuracy:", accuracy

c:\python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'rf' is not defined

In [223]:
#sending my final to a csv
c.to_csv("~/Desktop/c.csv" , sep=',', encoding='utf-8')

In [224]:
pd.read_csv("~/Desktop/c.csv", index_col=0)

,company,job_title,summary,avg,city,state,dumsal,visualization,manager,scientist,associate,machine,learning,engineer,senior,mining
5,Highlands Point Apartments,Data Scientist,"If you are a Data Scientist with experience, p...",125000.0,California,Cal,1,0,0,1,0,0,0,0,0,0
17,"Enrich Consulting, Inc.",Quantitative Modeler / Coder - Analyst,"As an Analyst, you will be customizing impleme...",70000.0,California,Cal,0,0,0,0,0,0,0,0,0,0
24,Negocios IT,Data scientist,*Required skills: 3-5 years’ of experience. ...,140000.0,California,Cal,1,0,0,0,0,0,0,0,0,0
32,NESS USA INC,Data Scientist,Background in applied statistical modeling on ...,105000.0,California,Cal,0,0,0,0,0,0,0,0,0,0
34,Protein Metrics Inc.,Customer Success Scientist,Our software allows scientists to use data the...,130000.0,California,Cal,1,0,0,1,0,0,0,0,0,0
46,Stria,Data Scientist,We are looking for an accomplished data scient...,110000.0,California,Cal,0,0,0,1,0,0,0,0,0,0
48,Intro Staffing,Software Engineer - Autonomous Driving / Self ...,Software Engineer - Autonomous Driving / Machi...,150000.0,California,Cal,1,0,0,0,0,1,1,1,0,0
51,Lore IO,Python Developer,"We are experts in search, analytics, machine l...",150000.0,California,Cal,1,0,0,0,0,1,1,0,0,1
66,FlowCommand Inc.,Data Scientist,Our sensors send ultrasonic data directly to o...,110000.0,California,Cal,0,0,0,1,0,0,0,0,0,0
93,Blue Owl,Actuarial Pricing Analyst,"We are a well funded team of elite developers,...",135000.0,California,Cal,1,0,0,1,0,0,0,0,0,0
